# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct, coalesce, lit

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [5]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [6]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [7]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [8]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   NULL|     BE|   NULL|    NULL|      1|  NULL|   269|  6|    69| NULL|       1|    NULL|    0.0|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070802| 1963| 1096|   NULL|     US|     TX|    NULL|      1|  NULL|     2|  6|    63| NULL|       0|    NULL|   NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|    NULL|
|3070803| 1963| 1096|   NULL|     US|     IL|    NULL|      1|  NULL|     2|  6|    6

## Algorithm: Same-state patent citations

1. **US patents with state**: Keep only patents from US with non-null, non-empty POSTATE (we can only assign same-state count when both citing and cited have state).
2. **Join citations with states**: Join citations with patent states twice — once to get the citing patent's state, once to get the cited patent's state.
3. **Same-state filter**: Keep only rows where Citing_State == Cited_State.
4. **Count per patent**: Group by citing patent and count; this gives same-state citation count per patent.
5. **Augment patents**: Left-join the full patents table with this count (fill missing with 0), then sort by SAME_STATE descending and take top 10.

In [9]:
patent_states = patents.filter(
    (col("COUNTRY") == "US") &
    col("POSTATE").isNotNull() &
    (col("POSTATE") != "")
)

citing_states = patent_states.select(
    col("PATENT").alias("CITING"),
    col("POSTATE").alias("Citing_State")
)
cited_states = patent_states.select(
    col("PATENT").alias("CITED"),
    col("POSTATE").alias("Cited_State")
)

joined = (citations
    .join(citing_states, "CITING", "inner")
    .join(cited_states, "CITED", "inner")
)
joined.show(5)

+-------+-------+------------+-----------+
|  CITED| CITING|Citing_State|Cited_State|
+-------+-------+------------+-----------+
|3070803|4483021|          MS|         IL|
|3070803|4133055|          NH|         IL|
|3070803|4921141|          CA|         IL|
|3070803|5557807|          FL|         IL|
|3070803|4484363|          CA|         IL|
+-------+-------+------------+-----------+
only showing top 5 rows



In [10]:
same_state = joined.filter(col("Citing_State") == col("Cited_State"))
same_state_count = same_state.groupBy("CITING").agg(count("*").alias("same_state_count"))

patents_augmented = (patents
    .join(same_state_count, patents.PATENT == same_state_count.CITING, "left")
    .select(patents["*"], same_state_count["same_state_count"])
    .withColumn("SAME_STATE", coalesce(col("same_state_count"), lit(0)).cast("int"))
    .drop("same_state_count")
)

top10 = patents_augmented.orderBy(col("SAME_STATE").desc()).limit(10)
top10.show()

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|SAME_STATE|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+----------+
|5959466| 1999|14515|   1997|     US|     CA|    5310|      2|  NULL|   326|  4|    46|  159|       0|     1.0|   NULL|  0.6186|    NULL|  4.8868|  0.0455|   0.044|    NULL|    NULL|       125|
|5983822| 1999|14564|   1998|     US|     TX|  569900|      2|  NULL|   114|  5|    55|  200|       0|   0.995|   NULL|  0.7201|    NULL|   12.45|     0.0|     0.0|    NULL|    NULL|       103|
|6008204| 1999|14606|   1998| 